In [1]:
import pandas as pd
import pm4py
from sklearn.preprocessing import LabelEncoder

### Based on filledLog

In [2]:
df_filledLog = pd.read_csv("bpi2017_filledLog.csv")

In [3]:
df_og = df_filledLog
df_og.head()

,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,treatment,successful,treatmentSuccess
0,Application_1000086665,1,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",...,22.0,True,241.28,False,0.0,5000.0,Offer_410892064,notTreated,0,0
1,Application_1000086665,1,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",...,22.0,True,241.28,False,0.0,5000.0,Offer_410892064,notTreated,0,0
2,Application_1000086665,1,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",...,22.0,True,241.28,False,0.0,5000.0,Offer_410892064,notTreated,0,0
3,Application_1000086665,1,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",...,22.0,True,241.28,False,0.0,5000.0,Offer_410892064,notTreated,0,0
4,Application_1000086665,1,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",...,22.0,True,241.28,False,0.0,5000.0,Offer_410892064,notTreated,0,0


In [4]:
df_og['treatment'] = df_og['treatment'].replace({'treated': 1, 'notTreated': 0}).astype('int64')
df_og['treatmentSuccess'] = df_og['treatmentSuccess'].replace({'Yes': 2, 'No': 1, '0': 0}).astype('int64')

In [19]:
df_og.rename(columns={'successful': 'caseSuccesful','treatment':'treatedCase' }, inplace=True)

Create column offerNumber

One can get per case the unique offerIDs and then set offerNumber 1-n accordingly.

In [14]:
#offer_id_number_mappings = {}
for case in df_og['case:concept:name'].unique():
    # Get the rows corresponding to the current case
    offer_ids = df_og[df_og['case:concept:name'] == case]['OfferID'].unique()

    # Initialize a counter to assign unique numbers
    offer_id_number = 1

    # Assign unique numbers to each OfferID
    for offer_id in offer_ids:
        df_og.loc[df_og['OfferID'] == offer_id, 'offerNumber'] = offer_id_number
        #offer_id_number_mappings[case, offer_id] = offer_id_number
        offer_id_number += 1
df_og

,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,treatedCase,successful,treatmentSuccess,offerNumber
0,Application_1000086665,1,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
1,Application_1000086665,1,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
2,Application_1000086665,1,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
3,Application_1000086665,1,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
4,Application_1000086665,1,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198314,Application_999993812,1,Obtained,User_41,W_Call incomplete files,Workflow,Workitem_1102967432,resume,2016-10-20 08:19:28.812000+00:00,Caravan / Camper,...,True,349.13,True,877.0,30000.0,Offer_913122697,0,1,0,1.0
1198315,Application_999993812,1,Released,User_41,W_Call incomplete files,Workflow,Workitem_454743075,suspend,2016-10-20 08:21:59.667000+00:00,Caravan / Camper,...,True,349.13,True,877.0,30000.0,Offer_913122697,0,1,0,1.0
1198316,Application_999993812,1,statechange,User_68,O_Accepted,Offer,OfferState_1927291643,complete,2016-10-24 06:24:30.056000+00:00,Caravan / Camper,...,True,349.13,True,877.0,30000.0,Offer_913122697,0,1,0,1.0
1198317,Application_999993812,1,statechange,User_68,A_Pending,Application,ApplState_1253134444,complete,2016-10-24 06:24:30.059000+00:00,Caravan / Camper,...,True,349.13,True,877.0,30000.0,Offer_913122697,0,1,0,1.0


In [17]:
df_og.to_csv('bpi2017_offerNumber.csv', index=False)

### Based on offerNumber dataset

In [2]:
df_og = pd.read_csv("bpi2017_offerNumber.csv")

In [3]:
df_og.head()

,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,treatedCase,successful,treatmentSuccess,offerNumber
0,Application_1000086665,1,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
1,Application_1000086665,1,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
2,Application_1000086665,1,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
3,Application_1000086665,1,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0
4,Application_1000086665,1,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",...,True,241.28,False,0.0,5000.0,Offer_410892064,0,0,0,1.0


In [4]:
df_og.rename(columns={'successful': 'caseSuccesful'}, inplace=True)

In [5]:
#trial code
df_og['offerSuccess'] = (df_og['caseSuccesful'] == 1) & (df_og['offerNumber'] == df_og['NumberOfOffers']).astype(int)
df_og['treatmentOffer'] = (df_og['offerNumber'] >= 2).astype(int)
df_og.head()

,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,Selected,CreditScore,OfferedAmount,OfferID,treatedCase,caseSuccesful,treatmentSuccess,offerNumber,offerSuccess,treatmentOffer
0,Application_1000086665,1,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",...,False,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0
1,Application_1000086665,1,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",...,False,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0
2,Application_1000086665,1,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",...,False,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0
3,Application_1000086665,1,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",...,False,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0
4,Application_1000086665,1,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",...,False,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0


In [6]:
#trial code
df_og["time:timestamp"] = pd.to_datetime(df_og["time:timestamp"], format='ISO8601')
df_og['timeApplication'] = df_og.groupby('case:concept:name')['time:timestamp'].transform(lambda x: x - x.min())
df_og['timeApplication'] = df_og['timeApplication'].dt.total_seconds()
df_og

,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,CreditScore,OfferedAmount,OfferID,treatedCase,caseSuccesful,treatmentSuccess,offerNumber,offerSuccess,treatmentOffer,timeApplication
0,Application_1000086665,1,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",...,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0,0.000
1,Application_1000086665,1,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",...,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0,0.061
2,Application_1000086665,1,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",...,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0,0.290
3,Application_1000086665,1,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",...,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0,66.613
4,Application_1000086665,1,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",...,0.0,5000.0,Offer_410892064,0,0,0,1.0,False,0,66.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198314,Application_999993812,1,Obtained,User_41,W_Call incomplete files,Workflow,Workitem_1102967432,resume,2016-10-20 08:19:28.812000+00:00,Caravan / Camper,...,877.0,30000.0,Offer_913122697,0,1,0,1.0,True,0,1189344.713
1198315,Application_999993812,1,Released,User_41,W_Call incomplete files,Workflow,Workitem_454743075,suspend,2016-10-20 08:21:59.667000+00:00,Caravan / Camper,...,877.0,30000.0,Offer_913122697,0,1,0,1.0,True,0,1189495.568
1198316,Application_999993812,1,statechange,User_68,O_Accepted,Offer,OfferState_1927291643,complete,2016-10-24 06:24:30.056000+00:00,Caravan / Camper,...,877.0,30000.0,Offer_913122697,0,1,0,1.0,True,0,1528045.957
1198317,Application_999993812,1,statechange,User_68,A_Pending,Application,ApplState_1253134444,complete,2016-10-24 06:24:30.059000+00:00,Caravan / Camper,...,877.0,30000.0,Offer_913122697,0,1,0,1.0,True,0,1528045.960


In [8]:
unique_values_count = df_og['org:resource'].nunique()
unique_values_count

148

In [10]:
#A_Create Application
smaller_df = df_og.head(20000)
def get_weekday(timestamp):
    return timestamp.weekday()

# Apply the function to create the new column 'weekdayApplication'
smaller_df['weekdayApplication'] = smaller_df[smaller_df['concept:name'] == 'A_Create Application']['time:timestamp'].apply(get_weekday)

# Forward fill the 'weekdayApplication' column to fill the NaN values for other rows in the same case
smaller_df['weekdayApplication'] = smaller_df.groupby('case:concept:name')['weekdayApplication'].transform(lambda x: x.ffill().bfill())
smaller_df

/scratch/slurm_tmpdir/job_23028212/ipykernel_163281/1141634729.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df['weekdayApplication'] = smaller_df[smaller_df['concept:name'] == 'A_Create Application']['time:timestamp'].apply(get_weekday)
/scratch/slurm_tmpdir/job_23028212/ipykernel_163281/1141634729.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df['weekdayApplication'] = smaller_df.groupby('case:concept:name')['weekdayApplication'].transform(lambda x: x.ffill().bfill())


,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,OfferedAmount,OfferID,treatedCase,caseSuccesful,treatmentSuccess,offerNumber,offerSuccess,treatmentOffer,timeApplication,weekdayApplication
0,Application_1000086665,1,Created,User_1,A_Create Application,Application,Application_1000086665,complete,2016-08-03 15:57:21.673000+00:00,"Other, see explanation",...,5000.0,Offer_410892064,0,0,0,1.0,False,0,0.000,2.0
1,Application_1000086665,1,statechange,User_1,A_Submitted,Application,ApplState_161925113,complete,2016-08-03 15:57:21.734000+00:00,"Other, see explanation",...,5000.0,Offer_410892064,0,0,0,1.0,False,0,0.061,2.0
2,Application_1000086665,1,Created,User_1,W_Handle leads,Workflow,Workitem_747707399,schedule,2016-08-03 15:57:21.963000+00:00,"Other, see explanation",...,5000.0,Offer_410892064,0,0,0,1.0,False,0,0.290,2.0
3,Application_1000086665,1,Deleted,User_1,W_Handle leads,Workflow,Workitem_1030261128,withdraw,2016-08-03 15:58:28.286000+00:00,"Other, see explanation",...,5000.0,Offer_410892064,0,0,0,1.0,False,0,66.613,2.0
4,Application_1000086665,1,Created,User_1,W_Complete application,Workflow,Workitem_1127124826,schedule,2016-08-03 15:58:28.293000+00:00,"Other, see explanation",...,5000.0,Offer_410892064,0,0,0,1.0,False,0,66.620,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Application_1032601860,2,statechange,User_37,O_Created,Offer,OfferState_1834088611,complete,2016-03-30 17:55:32.453000+00:00,Home improvement,...,12500.0,Offer_570390672,1,0,1,2.0,False,1,37516.657,2.0
19996,Application_1032601860,2,statechange,User_37,O_Cancelled,Offer,OfferState_1502782154,complete,2016-03-30 17:55:42.055000+00:00,Home improvement,...,12000.0,Offer_2006865323,1,0,1,1.0,False,0,37526.259,2.0
19997,Application_1032601860,2,statechange,User_37,O_Sent (mail and online),Offer,OfferState_1270964173,complete,2016-03-30 17:56:03.732000+00:00,Home improvement,...,12500.0,Offer_570390672,1,0,1,2.0,False,1,37547.936,2.0
19998,Application_1032601860,2,Deleted,User_37,W_Complete application,Workflow,Workitem_1317401156,ate_abort,2016-03-30 17:56:03.751000+00:00,Home improvement,...,12500.0,Offer_570390672,1,0,1,2.0,False,1,37547.955,2.0


In [11]:
df_og.to_csv('bpi2017_processed.csv', index=False)

Encode non-numeric columns to numeric

In [ ]:
non_numeric_columns = list(dataframe.select_dtypes(exclude=[np.number]).columns)

for col in non_numeric_columns:
    struct_data[col] = LabelEncoder().fit_transform(struct_data[col])

struct_data_type = struct_data.astype('float32')
struct_data_type.head(5)